## Code for Collecting dataset

In [1]:
import cv2
import os
import sys

IMG_SAVE_PATH = 'temp'
IMG_CLASS_PATH = os.path.join(IMG_SAVE_PATH, "five")

try:
    os.mkdir(IMG_SAVE_PATH)
except FileExistsError:
    pass
try:
    os.mkdir(IMG_CLASS_PATH)
except FileExistsError:
    print("{} directory already exists.".format(IMG_CLASS_PATH))
    print("All images gathered will be saved along with existing items in this folder")

cap = cv2.VideoCapture(0)

start = False
count = 0

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    if count == 500:
        break

    cv2.rectangle(frame, (380,80),(620,320), (255, 255, 255), 2)

    if start:
        roi = frame[80:320,380:620]
        save_path = os.path.join(IMG_CLASS_PATH, '{}.jpg'.format(count + 1))
        cv2.imwrite(save_path, roi)
        count += 1

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Collecting {}".format(count),
            (5, 50), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow("Collecting images", frame)

    k = cv2.waitKey(10)
    if k == ord('a'):
        start = not start

    if k == ord('q'):
        break

print("\n{} images saved to {}".format(count, IMG_CLASS_PATH))
cap.release()
cv2.destroyAllWindows()


500 images saved to temp\five


## Importing packages

In [7]:
import cv2
import numpy as np
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import tensorflow.keras.applications.mobilenet
import os

IMG_SAVE_PATH = r'dataset'


Str_to_Int = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    'none': 7
}

NUM_CLASSES = 8


def str_to_Int_mapper(val):
    return Str_to_Int[val]

## Defining Model

In [8]:
def get_model():
    model = Sequential([
        tf.keras.applications.mobilenet.MobileNet(input_shape=(240, 240, 3), include_top=False),
        Dropout(0.5),
        Convolution2D(8, (1, 1), padding='valid'),
        Activation('relu'),        # Rectified Linear Unit - Activation Function
        GlobalAveragePooling2D(),  # Pooling Layer
        Activation('softmax')      # Softmax - Activation Function - For Multi Class Classification
    ])
    return model

## Loading in the image data

In [9]:
dataset = []
for directory in os.listdir(IMG_SAVE_PATH):
    
    path = os.path.join(IMG_SAVE_PATH, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        new_path = os.path.join(path, item)
        for imgg in os.listdir(new_path):
            if item.startswith("."):
                continue
            img = cv2.imread(os.path.join(new_path, imgg))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (240, 240))
            dataset.append([img, item])

In [11]:
len(dataset)

28800

In [12]:
data, labels = zip(*dataset)
temp = list(map(str_to_Int_mapper, labels))

In [ ]:
temp

## Defining the model

In [22]:
# one hot encoding the labels
#labels = np_utils.to_categorical(temp)

import keras
labels = keras.utils.to_categorical(temp)

# define the model
model = get_model()
model.compile(
    optimizer=Adam(learning_rate=0.0001),         # lr = learning_rate
    loss='categorical_crossentropy',   # Multi-class classification
    metrics=['accuracy']  
)


In [23]:
np.array(data[0].shape)

array([240, 240,   3])

In [24]:
np.array(labels.shape)

array([28800,     8])

## Fitting the Model

In [26]:
# start training

model.fit(np.array(data), np.array(labels),verbose=2)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## Saving the Model

In [31]:
# save the model for later use
model.save("hand-cricket_new_new.h5")

## Model Counting Fingers

In [29]:
import tensorflow as tf

In [33]:
from tensorflow.keras.models import load_model


In [34]:
model = load_model(r'C:\Users\hamra\Desktop\ashhad\hand\HandCricketSimulation\model_VGG-10.h5')

In [41]:
model.input_shape

(None, 224, 224, 3)

In [36]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)

while True:
    return_value,frame = cap.read()
    frame = cv2.flip(frame,1)

    if not return_value:
        continue

    # cv2.rectangle(frame,(380,80),(620,320),(0,255,255),2)
    # hand_img = frame[80:320,380:620]
    # img = cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB)
    # pred = model.predict(np.array([img]))
    # cv2.putText(frame, "{}".format(np.argmax(pred)),(5, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
    # cv2.imshow("Hand Cricket", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

## Hand Cricket Simulation - Fast Version

In [1]:
import cv2
import numpy as np
from random import choice

NUM_TO_VALUE = {
    0: "Defense",
    1: "One",
    2: "Two",
    3: "Three",
    4: "Four",
    5: "Five",
    6: "Six",
    7: "None"
}

def mapper(val):
    return NUM_TO_VALUE[val]

In [2]:
from keras.models import load_model
model_1 = load_model('hand-cricket_new_new.h5')

In [ ]:
cap_1 = cv2.VideoCapture(0)
Batsman_Score = 0
High_Score = 0

while True:
    ret, frame = cap_1.read()
    if not ret:
        continue
    frame = cv2.flip(frame,1)
    
    # rectangle for user to play
    cv2.rectangle(frame,(380,80),(620,320),(0,255,255),2)
    
    # rectangle for computer to play
    cv2.rectangle(frame,(20,80),(260,320),(0,255,255),2)

    
    # extract the region of image within the user rectangle
    roi = frame[80:320,380:620]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

    
    # predict the move made
    computer_choices = ['zero', 'one', 'two', 'three', 'four', 'five', 'six']
    pred = model_1.predict(np.array([img]))
    move_code = np.argmax(pred)
    user_move_name = mapper(move_code)
    computer_move_name = choice(['zero', 'one', 'two', 'three', 'four', 'five', 'six'])
    
    
    # High Score Calculation
    High_Score = max(High_Score, Batsman_Score)
    
    
    # Hand Cricket Logic Implementation
    if move_code == 7:
        pass
    elif move_code == computer_choices.index(computer_move_name):
        Batsman_Score = 0
    elif move_code == 0:
        Batsman_Score += computer_choices.index(computer_move_name)
    else:
        Batsman_Score += move_code
    
    
    # display the information
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(frame, "Your Move: " + user_move_name,
                (370, 50), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (20, 50), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BOWLER",
                (75, 350), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BATSMAN",
                (440, 350), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "CURRENT SCORE: " + str(Batsman_Score),
                (180, 400), font, 0.8, (255, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "HIGH SCORE: " + str(High_Score),
                (180, 430), font, 0.8, (255, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow("Hand Cricket!", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        print("High Score: ", High_Score)
        break

cap_1.release()
cv2.destroyAllWindows()

## Hand Cricket Simulation - Slow Version

In [8]:
import cv2
import numpy as np
from random import choice

NUM_TO_VALUE = {
    0: "Defense",
    1: "One",
    2: "Two",
    3: "Three",
    4: "Four",
    5: "Five",
    6: "Six",
    7: "None"
}

def mapper(val):
    return NUM_TO_VALUE[val]

from keras.models import load_model
model_1 = load_model('hand-cricket_new_new.h5')

In [9]:
cap_1 = cv2.VideoCapture(0)
Batsman_Score = 0
High_Score = 0
fps = 0
user_moves = []
pred_moves = []
pred_values = []

user_move_name = "Initial"
computer_move_name = "Initial"

while True:
    ret, frame = cap_1.read()
    fps+=1
    if not ret:
        continue
    frame = cv2.flip(frame,1)
    
    # rectangle for user to play
    cv2.rectangle(frame,(380,80),(620,320),(0,255,255),2)
    
    # rectangle for computer to play
    cv2.rectangle(frame,(20,80),(260,320),(0,255,255),2)

    # extract the region of image within the user rectangle
    roi = frame[80:320,380:620]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    user_moves.append(img)
    
    # display the information
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(frame, "Your Move: ",
                (390, 50), font, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Previous Move: ",
                (20, 50), font, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "BOWLER",
                (75, 350), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "BATSMAN",
                (440, 350), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "CURRENT SCORE: ",
                (180, 400), font, 0.8, (255, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "HIGH SCORE: ",
                (180, 430), font, 0.8, (255, 0, 255), 1, cv2.LINE_AA)
    
    if fps==30:
        for i in user_moves:
            temp = np.array(i)
            pred_moves.append(model_1.predict(temp.reshape(1,240,240,3)))
        for i in pred_moves:
            pred_values.append(np.argmax(i))
        
        move_code = max(set(pred_values), key = pred_values.count)       
    
        # predict the move made
        computer_choices = ['zero', 'one', 'two', 'three', 'four', 'five', 'six']
        user_move_name = mapper(move_code)
        computer_move_name = choice(['zero', 'one', 'two', 'three', 'four', 'five', 'six'])
        
        
        fps = 0
        user_moves = []
        pred_moves = []
        pred_values = []
    
    
        # High Score Calculation
        High_Score = max(High_Score, Batsman_Score)


        # Hand Cricket Logic Implementation
        if move_code == 7:
            pass
        elif move_code == computer_choices.index(computer_move_name):
            Batsman_Score = 0
        elif move_code == 0:
            Batsman_Score += computer_choices.index(computer_move_name)
        else:
            Batsman_Score += move_code
    
    
    # display the information
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(frame, "Your Move: " + user_move_name,
                (390, 50), font, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Previous Move: " + computer_move_name,
                (20, 50), font, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "BOWLER",
                (75, 350), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "BATSMAN",
                (440, 350), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "CURRENT SCORE: " + str(Batsman_Score),
                (180, 400), font, 0.8, (255, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "HIGH SCORE: " + str(High_Score),
                (180, 430), font, 0.8, (255, 0, 255), 1, cv2.LINE_AA)
    
    cv2.imshow("Hand Cricket!", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        print("High Score: ", High_Score)
        break

cap_1.release()
cv2.destroyAllWindows()

High Score:  8
